## **Mini Project Document**
### 1. **Introduction**
#### 1.1. Team Member
+ Li Yitong
+ Qing Hanwen
+ Zhang Qin
+ Wang Zhiyou
+ Li Tianyi
#### 1.2. Programme
This jupyter notebook file showcase a python programme that automatically read information of students from a `.csv` file and assign each student a team of five within each tutorial group. The programme uses simulated annealing algorithm to make sure that every team allocation is fair and diverse. Eventually, the output is displayed in a new `.csv` file for result.

### 2. **I/O and Data Structure**
#### 2.1. For Each Student
Noticing that every student in the list has a lot of features to take into consideration, the thought of object-oriented programming(OOP) was introduced. Thus a class `Student` is defined to store every detail of each student. To reduce the difficluty of debugging, the class `Student` offers interfaces:
- `isDigit(target)` is designed to tell whether varible `target` is a digit or not. Then the function returns `True` or `False` accordingly.
- `getCGPA()` is an interface to get the object's `CGPA` value in `int` type.
- `getGender()` is an interface to get the corresponding value of the object's `gender` in the dictionary `genderTable`, which maps `"Male"` to `0` and `"Female"` to `1`.
- `getSchool()` is an interface to get the corresponding value of the object's `school` in the dictionary `schoolTable`, which maps every name of school to a unique value.
- `getTutorialGroup()` is an interface to get the number of a tutorial group since the group name is given in `string` form like `"G-114"`. This function goes through the `string` and extracts all the digits in it.
- `display()` is a method to let the object to show all critical values of itself in the console. 
#### 2.2. Input Stage
Given that the data of input includes 120 tutorial groups, a list `studentList` which contains 121 empty lists is defined. Therefore the index of `studentList` range form 0 to 120, we can use `list.append()` to add students to one of those empty lists with the identical index with their tutorial group. Especially, we use
```python
flag=True
while (...):
    if flag:
        flag=False
        continue
```
to skip the first row in the `.csv` file, which is the title row.
#### 2.3. Output Stage
The most important part in the output stage is the method to traverse all the students. Thanks to the design of `studentList`, that task can be very simple using the code below:
```python
for tutorialGroup in studentList:
    for student in tutorialGroup:
        [OUTPUT]
```
#### 2.4. File Operation
The programme needs to read the data it is going to process from `records.csv` stored in the folder `assets`. And it generates the final results in a new `.csv` file in the same folder as this Jupyter Notebook. To make sure the file streams are closed, `with open():` method is adopted in the code. Python itself contains a module named `csv` to deal with `.csv` files. All that need to do is `import csv`, and declear a `reader` or a `writer` instance to read or write files respectively.

### 3. **Design of Algorithm**
#### 3.1. Abstract
Basically, the task require us to consider three dimensions: the **gender**, **CGPA** and **school** of a student. And given that all the members in a team should come from the same tutorial group, then we can consider only one group and repeat the operation for 120 times. For each group, our thought is to first make sure that the gender consituent of each team is balanced. Therefore the question is transformed into a two-dimensional one. Then, an evaluation function is applied to quantify how average a group is, which returns higher value for worse result. According to the results given by the function, simulated annealing algortihm is used to geta best result. Finally, the programme outputs into a new `.csv` file.
#### 3.2. Distribution of Gender
Considering that handling with three types of values is a difficult task, the factor of gender is solved first. To achieve this, a new list, `tg[]`, is created which copies all the data of the tutorial group that is under operation now. The list `tg[]` is then sorted accoring to the `getGender()` value of each element. After that, the elements in `tg[]` is in order of gender and is distributed into 10 teams in this tutorial group in turns so that the gender feature in every team is balanced.
#### 3.3. Evaluation Function
In order to adopt simulated annealing method, every team allocation should be given a value showing how good or bad that list of member is. Therefore the algorithm can then decide how big the odd is to apply the new allocation rather than the old one. Noting that there are mutiple dimentions to be considered, the model of distance come up to mind. We use three diffrent expotional function to judge how distant they are to the ideal stage. The functions are designed intentionally so that all of the three values fall in a certain section. 
- For **school**, the function is $f(x)=\frac{1}{40}(x-1)e^x$ where x is the max number of people in a team that are from the same school.
- For **gender**, the function is $g(x)=\frac{1}{e}(x-c)e^{\alpha(x-c)}$ where x is the average gender value of a team at present and c is the overall average gender value of the tutorial group. $\alpha$ is a constant coefficient number decided by debugging.
- For **CGPA**, the function is $h(x)=\frac{1}{e}(x-c)e^{\gamma(x-c)}$ where x is the average CGPA value of a team at present and c is the overall average CGPA value of the tutorial group. $\gamma$ is a constant coefficient number decided by debugging.

Therefore, the final value given to a team is $V=\sqrt{f^2+g^2+h^2}$, just the same as calculating the difference. After knowing the `distance` of a team, the value from the function `distance()` is summed up in function `distanceGroup()` to get a total distance value of the whole tutorial group. 
#### 3.4. Simulated Annealing
![img_simulated annealing](.\assets\img\simulated-annealing.gif "Simulated Annealing Example")
> Simulated annealing (SA) is a probabilistic technique for approximating the global optimum of a given function. Specifically, it is a metaheuristic to approximate global optimization in a large search space for an optimization problem. For large numbers of local optima, SA can find the global optimum.

From Wikipedia, it is known that simulated annealing is capable of dealing with large amount of data and can find the best answer globally. Because this algorithm introduces radom factors, the possibility of getting better answer increases in a large scale. In this specific case, two students from different teams are selected and swapped. The programme then compares the value of function `distanceGroup()` before and after the exchange happens. Do note that the function value is recorded in two varibles $pre$ and $aft$ accordingly. If the new allocation of team is better, it is accepted. Otherwise, there is an odd of $P=1-e^{\frac{pre-aft}{T}}$ to accept the result that is not as good as the present one, where $T$ is the temperature at the time of operation. Finally, at the end of the loop, the temperature $T$ is cooled by a ratio of $f=0.995$. The loop continues working until the temperature is lowered below $0.001$, exactly when the result stablizes around the best answer.

### 4. **Flow Chart**
![img]()
### 5. **Challenges And Issues**
#### 5.1 Challenges
Originally, the plan was to use a three dimentional evaluation function and do simulated annealing directly. However, the result was always neither stable nor fair enough to meet the requirements. Moreover, the process of coding and debugging also contains a lot of difficulties to cope with bugs and logical fallacies. Another challenge is the adjustment of the coefficient number to give into the function `distanceGroup()`, which directly affects the quality of the final result. But after careful dubugging and productive discussion as a team, the difficulties are eventually overcomed.
#### 5.2 Issues
This programme can only deal with allocation with teams of five at present. There would be some dificulties to adjust the code to enable it to make teams with given number of members. And to make sure the result is good enough to meet the requirements, the ramdom adjustments was carried out for a great number of times. It causes our programme run a little bit slowly. In other words, the value of temperature $T$ and anneal ratio $f$ could have the possibility to get improved.

### 6. **Conclusion**
Generally speaking, this programme gives a primary measure to separate students in a tutorial group into teams of five considering both fairness and diversity. And with the use of simulated annealing to generate result, the stability of the final output can be guaranteed. In addition, compared to static algorithms, our result is different every time the programme is run instead of giving , which may offer another kind of fairness.

In [1]:
import csv
from random import *

In [2]:
#Give the gender strings and school strings certain value
#Then how good a list is can be distinguished by function "distance()"

genderTable={
    "Male":0,
    "Female":1
}
schoolTable={
    "CCDS":1,
    "EEE":2,
    "CoB (NBS)":3,
    "SoH":4,
    "WKW SCI":5,
    "CoE":6,
    "MAE":7,
    "SPMS":8,
    "SBS":9,
    "SSS":10,
    "ASE":11,
    "NIE":12,
    "ADM":13,
    "MSE":14,
    "LKCMedicine":15,
    "CCEB":16,
    "CEE":17,
    "HASS":18
}

In [3]:
#Use class to store the data of a student
#Define some interface for outside functions to get values

class Student:
    def __init__(self,gender,school,CGPA,tutorialGroup,ID,name):
        self.gender=gender
        self.school=school
        self.CGPA=CGPA
        self.tutorialGroup=tutorialGroup
        self.name=name
        self.ID=ID
        self.team=""
    def isDigit(self,target):
        return ord(target) in range(ord('0'),ord('9')+1)
    def getCGPA(self):
        return float(self.CGPA)*100
    def getGender(self):
        return genderTable[self.gender]
    def getSchool(self):
        return schoolTable[self.school]
    def getTutorialGroup(self):
        res=0
        for i in self.tutorialGroup:
            if self.isDigit(i):
                res=res*10+ord(i)-ord('0')
        return res
    def display(self):
        print(self.tutorialGroup,self.school,self.CGPA,self.gender,self.team)

In [4]:
#For every tutorial group, the extent of their evenness "distanceGroup()" is calculated by a sum of "distance()" of each team.

def distance(targetGender:float,targetCGPA:float,presentList:list[Student],coefficientGender=1,coefficientCGPA=1,coefficientSchool=1):
    presentGender=0
    presentCGPA=0
    schoolRecord=[0 for i in range(0,19)]
    maxSame=0
    for student in presentList:
        schoolRecord[student.getSchool()]+=1
        presentCGPA+=student.getCGPA()
        presentGender+=student.getGender()
    maxSame=max(schoolRecord)
    distGender=(1/(2.71828))*(((presentGender/5.0)-targetGender))*2.71828**(coefficientGender*((presentGender/5.0)-targetGender))
    distCGPA=(1/(2.71828))*((presentCGPA/5.0)-targetCGPA)*2.71828**(coefficientCGPA*((presentCGPA/5.0)-targetCGPA))
    distSchool=(1/40.0)*(maxSame-1)*2.71828**maxSame

    return coefficientGender*distGender**2+coefficientCGPA*distCGPA**2+coefficientSchool*(distSchool)**2

def distanceGroup(groupList:list[Student],coefficientGender=1,coefficientCGPA=1,coefficientSchool=1):
    targetGender=0
    targetCGPA=0
    for student in groupList:
        targetGender+=student.getGender()
        targetCGPA+=student.getCGPA()
    targetGender/=50.0
    targetCGPA/=50.0
    groupDistance=0
    for i in range(0,10):
        teamList:list[Student]=[]
        for j in range(5*i,5*i+5):
            teamList.append(groupList[j])
        groupDistance+=distance(targetGender,targetCGPA,teamList,coefficientGender,coefficientCGPA,coefficientSchool)
    return groupDistance


In [5]:
#Read file
studentList:list[list[Student]]=[[] for i in range(0,121)]
with open("./assets/records.csv","r",encoding="utf-8") as records:
    reader=csv.reader(records)
    flag=True
    for row in reader:
        if flag:
            flag=False
            continue
        present=Student(row[4],row[2],row[5],row[0],row[3],row[1])
        studentList[present.getTutorialGroup()].append(present)

In [6]:
#Algorithm runs here

teamID=1 #Global varible to give each team a unique number
flag=True
for tutorialGroup in studentList:
    #Skip the title of record.csv
    if flag: 
        flag=False
        continue
    
    #Pre-allocate the list according to gender    
    tg=[]
    for student in tutorialGroup:
        tg.append(student)
    tg.sort(key=lambda x:x.getGender())
    i=0
    for i in range(50):
        tutorialGroup[i%10*5+int(i/10)]=tg[i]
    
    #Simulated annealing algorithm for CGPA and School
    T=10000.0
    f=0.995
    maxScore=-1145141919810
    presentList:list[Student]=[]
    while T>0.001:
        p1=0
        p2=0
        while p1%5==p2%5 or tutorialGroup[p1].getGender()!=tutorialGroup[p2].getGender():
            p1=randint(0,49)
            p2=randint(0,49)
        pre=-distanceGroup(tutorialGroup,4,1,0.5)
        #Swap
        tutorialGroup[p1],tutorialGroup[p2]=tutorialGroup[p2],tutorialGroup[p1]
        aft=-distanceGroup(tutorialGroup,4,1,0.5)
        if aft>maxScore:
            maxScore=aft
            presentList=[]
            for i in tutorialGroup:
                presentList.append(i)
        if pre>aft and random()>(2.71828**((aft-pre)/T)):
            #Swap back
            tutorialGroup[p1],tutorialGroup[p2]=tutorialGroup[p2],tutorialGroup[p1]
        T*=f
    tutorialGroup[:]=presentList
    for i in range(0,10):
        for j in range(5*i,5*i+5):
            tutorialGroup[j].team=teamID
        teamID+=1
    print(f"G-{tutorialGroup[0].getTutorialGroup()} allocated successfully!")

#Display final result in console
# i=0
# for tutorialGroup in studentList:
#     for student in tutorialGroup:
#         student.display()
#         i+=1
#         if i%5==0:
#             print()

G-1 allocated successfully!
G-2 allocated successfully!
G-3 allocated successfully!
G-4 allocated successfully!
G-5 allocated successfully!
G-6 allocated successfully!
G-7 allocated successfully!
G-8 allocated successfully!
G-9 allocated successfully!
G-10 allocated successfully!
G-11 allocated successfully!
G-12 allocated successfully!
G-13 allocated successfully!
G-14 allocated successfully!
G-15 allocated successfully!
G-16 allocated successfully!
G-17 allocated successfully!
G-18 allocated successfully!
G-19 allocated successfully!
G-20 allocated successfully!
G-21 allocated successfully!
G-22 allocated successfully!
G-23 allocated successfully!
G-24 allocated successfully!
G-25 allocated successfully!
G-26 allocated successfully!
G-27 allocated successfully!
G-28 allocated successfully!
G-29 allocated successfully!
G-30 allocated successfully!
G-31 allocated successfully!
G-32 allocated successfully!
G-33 allocated successfully!
G-34 allocated successfully!
G-35 allocated successf

In [7]:
#Write the processed list into a new .csv file
with open("output.csv","w",encoding="utf-8") as output:
    writer=csv.writer(output)
    writer.writerow(["Tutorial Group","Student ID","School","Name","Gender","CGPA","Team Assigned"])
    for tutorialGroup in studentList:
        for student in tutorialGroup:
            writer.writerow([student.tutorialGroup,student.name,student.school,student.ID,student.gender,student.CGPA,student.team])

### **Use of AI Tools in Project Work**

Each team member should indicate either A or B.
1. I affirm that my contribution(s) to the lab work is my own, produced without help from any AI tool(s)
2. I affirm that my contribution(s) to the lab work has been produced with the help from AI tool(s)

|Full Name|Date|A or B|
|--------------------|-|-|
|Qing Hanwen|12/11/2025|A|
|Li Yitong|||
|Zhang Qin|||
|Wang Zhiyou|||
|Li Tianyi|||